In [34]:
from owlready2 import *
import re


In [35]:
#PascalCase
def to_pascal_case(text):
    text = text.replace('_', ' ').replace('-', ' ')
    words = text.split()
    return ''.join([word.capitalize() for word in words])
#camelCase
def to_camel_case(prop_name):
    words = prop_name.replace("_", " ").split()
    return words[0].lower() + "".join(word.capitalize() for word in words[1:]) if words else ""


def get_axiom(class_name, ontology, type = ["equivalent_to", "general_class_axiom", "sub_class", "disjoint"]):
    
    if type == "equivalent_to":
        return ontology[class_name].equivalent_to[0]
    elif type == "general_class_axiom":
        gcas = list(ontology.general_class_axioms())
        for gca in gcas:
            if gca.is_a[0] == getattr(ontology, class_name): 
                #print("GCA",gca.left_side)
                return gca.left_side
            else:
                pass
        
                
    elif type == "sub_class":
        print(ontology[class_name].is_a)
        return ontology[class_name].is_a[0]
    
    elif type == "disjoint":
        #print(*class_name.disjoints())
        return ontology[class_name].disjoints()


def axiom_to_puml(class_name, ontology, c_type):
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    puml_output = []
    puml_output.append("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml")
    class_map = {}
    counter = 1

    def get_class_name(entity):
        """Extracts and maps class names with proper prefixes."""
        nonlocal counter
        if isinstance(entity, ThingClass):
            #print(counter, 'processing', entity, entity.label[0])
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            class_name = entity.name
            if class_name not in class_map:
                class_map[class_name] = f"c{counter}"
                puml_output.append(f'class({class_map[class_name]}, {prefix}{to_camel_case(entity.label[0]) if class_name.startswith("BFO") else class_name})')
                counter += 1
            # print(puml_output)
            return class_map[class_name]

        elif isinstance(entity, Restriction):
            if isinstance(entity.property, Inverse):    
                # print(counter)
                #print('Restriction', entity.property.property.label[0])
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(entity.property.property.label[0]) if entity.property.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                # print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
            else:
                # print(counter)
                # print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(entity.property.label[0]) if entity.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                # print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
        
        elif isinstance(entity, And): 
            # print('And', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Or): 
            # print('Or', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Not):
            print('Not', entity)
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Inverse):
            print('Inverse', entity.property)
            return

        else:
            print('error',entity)
            raise ValueError(f"Unsupported entity type: {type(entity)}")

    def process_restriction(restriction, prop_name):
        """Handles OWL 'some' restrictions including unions inside."""
        nonlocal counter
        node = None
        if isinstance(restriction, And):
            entities = [get_class_name(e) for e in restriction.Classes]
            node = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"intersection({node}, '[{entity_list}]')")
            counter += 1
        elif isinstance(restriction, Or):
            entities = [get_class_name(e) for e in restriction.Classes]
            node = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"union({node}, '[{entity_list}]')")
            counter += 1
        elif isinstance(restriction, Not):
            entities = [get_class_name(restriction.Class)]
            node = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"complement({node}, {entity_list})")
            counter += 1
        elif hasattr(restriction, 'value') and isinstance(restriction.value, Or):
            entities = [get_class_name(e) for e in restriction.value.Classes]
            node = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"union({node}, '[{entity_list}]')")
            counter += 1
        else:
            # Default: assume restriction.value is a class
            class_name = get_class_name(restriction.value)
            # If no property is provided, return the class node directly.
            if prop_name is None:
                return class_name
            else:
                node = class_name

        # Only wrap the node in a "some" restriction if a property name is given.
        if prop_name is not None:
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {node})")
            counter += 1
            return var_name
        else:
            return node
    


    axiom = get_axiom(class_name, ontology, c_type)
    class_entity = ontology[class_name]
    main_class_name = get_class_name(class_entity)
    processed_parts = [get_class_name(part) for part in axiom.Classes]
    #change from classes to axiom and let the func figure it out
    # print(processed_parts)

    if len(processed_parts) > 1:
        final_union = f"ce{counter}"
        entity_list = ", ".join(f'"{e}"' for e in processed_parts)
        puml_output.append(f"union({final_union}, '[{entity_list}]')")
        puml_output.append(f"equivalent({main_class_name}, {final_union})")

    return "\n".join(puml_output)




In [38]:
# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity], [axiom, ActionSpecification, EncodedAlgorithm, CommercialServiceAgreement, PlanSpecification]
input = "ProcuringBusinessProcess"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
print(iofonto[input].equivalent_to[0])
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(input, iofonto, "equivalent_to")

print(axiom_result)

Core.BusinessProcess & obo.BFO_0000117.some(Core.BuyingBusinessProcess) & obo.BFO_0000117.some(Core.SellingBusinessProcess | Core.SupplyingBusinessProcess)
!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml
class(c1, iof:ProcuringBusinessProcess)
class(c2, iof:BusinessProcess)
class(c3, iof:BuyingBusinessProcess)
some(ce4, bfo:hasOccurrentPart, c3)
class(c5, iof:SellingBusinessProcess)
class(c6, iof:SupplyingBusinessProcess)
union(ce7, '["c5", "c6"]')
some(ce8, bfo:hasOccurrentPart, ce7)
union(ce9, '["c2", "ce4", "ce8"]')
equivalent(c1, ce9)


In [7]:
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
input = "MeasurementInformationContentEntity"
axiom = iofonto[input].equivalent_to[0]
print(axiom)

Core.InformationContentEntity & obo.BFO_0000110.some(Core.MeasuredValueExpression) & Core.describes.some(obo.BFO_0000008 | obo.BFO_0000020 | Core.ProcessCharacteristic) & Core.isAbout.some(obo.BFO_0000015 | obo.BFO_0000035 | (obo.BFO_0000004 & Not(obo.BFO_0000006))) & Core.isOutputOf.some(Core.MeasurementProcess)


In [ ]:
#v1 (bug: does not handle AND, OR, NOT properly)
#PascalCase
def to_pascal_case(text):
    text = text.replace('_', ' ').replace('-', ' ')
    words = text.split()
    return ''.join([word.capitalize() for word in words])
#camelCase
def to_camel_case(prop_name):
    words = prop_name.replace("_", " ").split()
    return words[0].lower() + "".join(word.capitalize() for word in words[1:]) if words else ""


def get_axiom(class_name, ontology, type = ["equivalent_to", "general_class_axiom", "sub_class", "disjoint"]):
    
    if type == "equivalent_to":
        return ontology[class_name].equivalent_to[0]
    elif type == "general_class_axiom":
        gcas = list(ontology.general_class_axioms())
        for gca in gcas:
            if gca.is_a[0] == getattr(ontology, class_name): 
                #print("GCA",gca.left_side)
                return gca.left_side
            else:
                pass
        
                
    elif type == "sub_class":
        print(ontology[class_name].is_a)
        return ontology[class_name].is_a[0]
    
    elif type == "disjoint":
        #print(*class_name.disjoints())
        return ontology[class_name].disjoints()


def axiom_to_puml(class_name, ontology, c_type):
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    puml_output = []
    puml_output.append("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml")
    class_map = {}
    counter = 1

    def get_class_name(entity):
        """Extracts and maps class names with proper prefixes."""
        nonlocal counter
        if isinstance(entity, ThingClass):
            #print(counter, 'processing', entity, entity.label[0])
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            class_name = entity.name
            if class_name not in class_map:
                class_map[class_name] = f"c{counter}"
                puml_output.append(f'class({class_map[class_name]}, {prefix}{to_camel_case(entity.label[0]) if class_name.startswith("BFO") else class_name})')
                counter += 1
            # print(puml_output)
            return class_map[class_name]

        elif isinstance(entity, Restriction):
            if isinstance(entity.property, Inverse):    
                # print(counter)
                #print('Restriction', entity.property.property.label[0])
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(entity.property.property.label[0]) if entity.property.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                # print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
            else:
                # print(counter)
                # print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(entity.property.label[0]) if entity.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                # print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
        
        elif isinstance(entity, And): 
            # print('And', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Or): 
            # print('Or', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Not):
            print('Not', entity)
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Inverse):
            print('Inverse', entity.property)
            return

        else:
            print('error',entity)
            raise ValueError(f"Unsupported entity type: {type(entity)}")

    def process_restriction(restriction, prop_name):
        """Handles OWL 'some' restrictions including unions inside."""
        nonlocal counter
        if prop_name is None:
           print('none propname',restriction)
           

        if isinstance(restriction, And):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"intersection({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        elif isinstance(restriction, Or):
            print('or', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        elif isinstance(restriction, Not):
            entities = [get_class_name(restriction.Class)]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities) 
            puml_output.append(f"complement({union_name}, {entity_list})")
            counter += 1
            # var_name = f"ce{counter}"
            # puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            # counter += 1
            return union_name

        elif isinstance(restriction.value, Or):
            print('re.val.or', restriction.value)
            entities = [get_class_name(e) for e in restriction.value.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities) 
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        else:  
            print('restriction else', restriction)
            class_name = get_class_name(restriction.value)
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {class_name})")
            counter += 1
            # print(puml_output)
            return var_name
    


    axiom = get_axiom(class_name, ontology, c_type)
    class_entity = ontology[class_name]
    main_class_name = get_class_name(class_entity)
    processed_parts = [get_class_name(part) for part in axiom.Classes]
    #change from classes to axiom and let the func figure it out
    # print(processed_parts)

    if len(processed_parts) > 1:
        final_union = f"ce{counter}"
        entity_list = ", ".join(f'"{e}"' for e in processed_parts)
        puml_output.append(f"union({final_union}, '[{entity_list}]')")
        puml_output.append(f"equivalent({main_class_name}, {final_union})")

    return "\n".join(puml_output)
# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity], [axiom, ActionSpecification, EncodedAlgorithm, CommercialServiceAgreement, PlanSpecification]
input = "ProcuringBusinessProcess"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
print(iofonto[input].equivalent_to[0])
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(input, iofonto, "equivalent_to")

print(axiom_result)

